# Lacunas Estruturais

Importando bibliotecas:

In [1]:
import pandas as pd
import networkx as nx
import socnet as sn

from random import shuffle
from itertools import combinations

Configurando bibliotecas:

In [2]:
sn.node_size = 5

sn.edge_width = 1
sn.edge_color = (191, 191, 191)

Definindo constantes:

In [24]:
FRAC_CLOSERS = 0.80

NUM_NODES = 25

DENSITY = 0.5

OPENER = 0 #busca lacuna estrutural
CLOSER = 1 #não quer lacuna estrutural

OPENER_COLOR = (255, 0, 0)
CLOSER_COLOR = (0, 0, 255)

Exemplo de grafo:

(esse grafo representa o *ambiente* do modelo)

In [25]:
def build_graph():
    g = nx.erdos_renyi_graph(NUM_NODES, DENSITY)
    #gera grafos aleatórios, 
    #que inclui o limite entre cada par de nós com igual probabilidade, 
    #independentemente das extremidades

    sn.reset_node_colors(g)
    sn.reset_edge_colors(g)

    return g


g = build_graph()

sn.reset_positions(g)

sn.show_graph(g)

Exemplo de distribuição de tipos de nó:

(esses tipos representam os *agentes* do modelo)

In [26]:
def set_types(g, num_closers=0):
    nodes = list(g.nodes)
    shuffle(nodes)

    for n in nodes[num_closers:]:
        g.nodes[n]['type'] = OPENER

    for n in nodes[:num_closers]:
        g.nodes[n]['type'] = CLOSER

    for n in g.nodes:
        if g.nodes[n]['type'] == OPENER:
            g.nodes[n]['color'] = OPENER_COLOR
        else:
            g.nodes[n]['color'] = CLOSER_COLOR


set_types(g, int(NUM_NODES / 2))

sn.show_graph(g)

Custo de manter uma relação do nó $n$:

$$c_n = \left\{\begin{array}{cl}
0             & \textrm{se }d_n = 0\textrm{;} \\
\frac{1}{d_n} & \textrm{caso contrário,} \\
\end{array}\right.$$

onde $d_n$ é o grau de $n$.

In [27]:
def cost(g, n):
    dn = g.degree(n)

    if dn == 0:
        return 0
    return 1 / dn

Restrição do nó $n$ para *openers* (fórmula de Burt):

$$\sum_{m \in N_n} \left(c_n + \sum_{k \in N_n, k \in N_m} c_n c_k\right)^2\textrm{,}$$

onde $N_n$ é o conjunto de vizinhos de $n$.

Esta fórmula parte da premissa de que o nó não está isolado.

Interpretação informal: o agente busca aumentar suas conexões e prefere conexões indiretas.

In [48]:
def opener_constraint(g, n):
    constraint = 0

    for m in g.neighbors(n):
        sub_constraint = cost(g, n)

        for k in g.neighbors(n):
            if g.has_edge(k, m):
                sub_constraint += cost(g, n) * cost(g, k)

        constraint += sub_constraint**2
    print(constraint)
    return constraint

Restrição do nó $n$ para *closers* (uma fórmula alternativa):

$$\frac{9}{8} - b_n\textrm{,}$$

onde $b_n$ é a fórmula de Burt.

Esta fórmula parte da premissa de que o nó não está isolado.

Interpretação informal: o agente busca diminuir suas conexões e prefere conexões diretas.

In [49]:
def closer_constraint(g, n):
    constraint = 0

    for m in g.neighbors(n):
        sub_constraint = cost(g, n)

        for k in g.neighbors(n):
            if g.has_edge(k, m):
                sub_constraint += cost(g, n) * cost(g, k)

        constraint += sub_constraint**9
    print(constraint)
    return constraint

Restrição geral do nó $n$:

* se o nó estiver isolado, é igual a $2$;
* caso contrário, depende de qual é o tipo.

In [50]:
def constraint(g, n):
    if g.degree(n) == 0:
        return 2

    if g.nodes[n]['type'] == OPENER:
        return opener_constraint(g, n)
    return closer_constraint(g, n)

Inicialização da simulação:

In [51]:
def initialize(g):
    for n in g.nodes:
        g.nodes[n]['constraint'] = constraint(g, n)

Passo da simulação:

1. escolha aleatoriamente um par de nós;
2. considere uma *inversão* da situação desse par, ou seja, remover a aresta se ela existir ou adicionar se não existir;
3. calcule o *ganho* de cada um desses nós, ou seja, a redução da restrição como consequência dessa inversão;
4. no caso particular em que a inversão foi adição, se um dos ganhos for negativo, os dois ganhos são redefinidos como zero *(justificativa no código)*;
5. se um dos ganhos foi positivo aceite a inversão e termine o passo, senão volte para (1);
6. se todos os pares foram escolhidos e nenhuma inversão foi aceita, termine o passo.

In [52]:
def invert(g, n, m):
    if g.has_edge(n, m):
        g.remove_edge(n, m)
    else:
        g.add_edge(n, m)
        g.edges[n, m]['color'] = sn.edge_color


def update(g):
    # Para cada par de nós, em ordem aleatória.

    pairs = list(combinations(g.nodes, 2))
    shuffle(pairs)

    for n, m in pairs:
        # Inverte a situação do par.
        invert(g, n, m)

        # Calcula as restrições após inversão.
        constraint_n = constraint(g, n)
        constraint_m = constraint(g, m)

        # Calcula os ganhos após inversão,
        # (lembrando que restrição é ruim)
        gain_n = g.nodes[n]['constraint'] - constraint_n
        gain_m = g.nodes[m]['constraint'] - constraint_m

        # Se a inversão foi adição e um dos nós foi prejudicado,
        # então o ganho é zero porque essa inversão é impossível.
        # (você não pode forçar uma pessoa a manter uma relação)
        if g.has_edge(n, m) and (gain_n < 0 or gain_m < 0):
            gain_n = 0
            gain_m = 0

        # Se algum nó teve um ganho, aceita a inversão e
        # considera o passo da simulação como terminado,
        # devolvendo True para indicar que houve mudança.
        if gain_n > 0 or gain_m > 0:
            g.nodes[n]['constraint'] = constraint_n
            g.nodes[m]['constraint'] = constraint_m
            return True

        # Se nenhum nó teve um ganho, essa inversão não é aceita.
        # Restauramos, portanto, a situação original do par de nós.
        invert(g, n, m)

    # Se passamos por todos os pares e nenhuma inversão foi aceita,
    # considera o passo da simulação como terminado, devolvendo
    # False para indicar que não houve mudança: a rede estabilizou.
    return False

Simulação com visualização, quando todos os nós buscam reduzir a restrição de Burt:

In [53]:
def simulate(num_closers):
    g = build_graph()

    set_types(g, num_closers)

    initialize(g)

    while update(g):
        pass

    return g


num_closers = 0

g = simulate(num_closers)

sn.reset_positions(g)

sn.show_graph(g)

0.17644967044915627
0.231299768046758
0.15712641328853144
0.1527050499259085
0.1711236934022485
0.15018925072769662
0.16146957075451576
0.1859988856830175
0.19598611624753454
0.18255672229889167
0.1669158808650737
0.17389927957313236
0.15945820446099243
0.1622758181659388
0.17747249279314245
0.17906764970142058
0.15812779287812576
0.1707863956834257
0.17679975468461687
0.17456727761489504
0.19206660028075162
0.16061956512673123
0.16236919669018346
0.1677894800052761
0.16971319330524812
0.16813871209763626
0.18687472857909762
0.15955608877985133
0.1632729720223596
0.15564926444574842
0.17313632113317154
0.18741788669226223
0.1676865149174845
0.16360270833887666
0.16533515275632535
0.16422907797663797
0.18811589973588438
0.18862510208848696
0.1932994282141493
0.25558528817863235
0.17000801536167437
0.18204170851545326
0.16630368187949146
0.1757134206764756
0.16063695204456313
0.175559840512142
0.16384266737028955
0.16446688736911708
0.18480419125433242
0.16450462571260296
0.1668151451552

0.15245654781425158
0.14823852279747943
0.16273581388029595
0.14599130999436014
0.15827123316319866
0.18137448060362052
0.14558409552010787
0.1447827605403053
0.16159795814317468
0.1505901838557552
0.16131152310753963
0.14274899308557154
0.15273769051808694
0.1438228709980428
0.15225813886578995
0.14810402569313333
0.15297381238270102
0.1634940453169552
0.15122029410471188
0.14372470512994268
0.1629438429713216
0.16156958952780642
0.13936703352259044
0.1566268669436895
0.1427534169188218
0.1539469621900113
0.14716573622569643
0.1536340573476723
0.14834401176030207
0.17161563202670596
0.14842820714122076
0.1426490436798004
0.1538862615073903
0.1534298356429106
0.14545968952567964
0.154695223765665
0.16165195650527758
0.15371290822008468
0.16406952514469436
0.17764042027626095
0.1517158360068072
0.16809979053985696
0.14296352607454832
0.1415731604020783
0.1777823222336272
0.15345471093322194
0.14432952715466654
0.13975049424509264
0.14422215568101254
0.1592203345971032
0.1472808789568092

0.1455701846429976
0.1469466840234036
0.15949836062777484
0.1442626407657451
0.16124488272170995
0.14044839095673256
0.14776140464127263
0.13257934025631654
0.14959820393157466
0.14922298478309523
0.14624624203019135
0.14726008662049178
0.14366222454608477
0.14581527686871235
0.1457483618424305
0.14321308228572563
0.15140589866888712
0.15637025508711544
0.1382989148262937
0.15176315445676705
0.1427114495854579
0.14718508813553838
0.151112482048011
0.15146580072915777
0.15255037191898446
0.151590057067644
0.14496324477895045
0.15973550842811365
0.15475336493852002
0.15842407552156937
0.15350660946577846
0.15974452206782375
0.13784215941886396
0.155220273279156
0.14837237037584122
0.149050445374848
0.1535187028566024
0.14333545833925007
0.14700371057968248
0.1601342380089089
0.1463406496440129
0.16569030572411414
0.15372777687778805
0.13573988317432373
0.14967626227449732
0.1516675918828245
0.15480623401135124
0.13294264321190521
0.1516588629214669
0.14190859009147383
0.1428892081323969


0.12694459739912162
0.11145569757931353
0.125
0.13348289397067065
0.11429801604485139
0.13576982832352463
0.11717043013275705
0.12004919193730385
0.1353387741046832
0.13977442497703918
0.1334782831063917
0.1499030199867904
0.14360738088007372
0.11489391996975848
0.12714992854852997
0.12151195785936048
0.13458307037657086
0.12386092745638203
0.1509915103233699
0.12366130844631681
0.14171602779811376
0.14617943067033978
0.14026848569630168
0.10363693755739212
0.1603931333575127
0.1314162794680714
0.10928324462465941
0.11954815937753005
0.13713078426365138
0.10092293247189316
0.14938706777025865
0.11322401554830003
0.1620865824734706
0.1254118655548129
0.1287573753884526
0.11905812267299738
0.15117872566439003
0.12739149537457134
0.12890783847077555
0.10496349250919751
0.13886304696578444
0.1274657507312179
0.1274657507312179
0.13499017957351292
0.11201540934568326
0.14692874225341762
0.12010524391310273
0.11211674829099075
0.1342064593631545
0.12521873146464413
0.15668686483497019
0.1472

0.13374791156514643
0.11436566972697508
0.10506435254614158
0.1553467513461514
0.13072910242407415
0.13912649368859828
0.10246867545818594
0.16009032617193053
0.12011608141484463
0.1308568063637994
0.1212634394497059
0.16265083520036902
0.12425284862988345
0.14798447545442286
0.11793739148645724
0.11148606028713592
0.11398709730440501
0.10363693755739214
0.14812529545047032
0.0955054012345679
0.1173316456155174
0.1668371466140697
0.12086994296711664
0.1261771154253672
0.12768270555202374
0.12186343054823534
0.14516563514121744
0.09407943067033976
0.12378952668834667
0.14445246137429696
0.12625264261017566
0.12159292621013193
0.12011608141484463
0.1311061971245538
0.11801970026800984
0.12028388235086496
0.1080903412129878
0.15604953145321868
0.09539142970633356
0.14473426499332412
0.12186812124894879
0.1398684081339967
0.13965486234883506
0.13480734198673666
0.13072277261174783
0.12712671159197772
0.09391482442759679
0.16382716092240107
0.10579492415379331
0.12099378702734194
0.12061461

0.12123874194095641
0.1212387419409564
0.09690964870341279
0.10005440419963914
0.11103497926690359
0.11334607125533655
0.10918547681681243
0.11106582908156337
0.10505310356663432
0.10349845958237569
0.14169105014193364
0.10803964748786203
0.08837546870217326
0.12825555863835295
0.11023197767001305
0.11585843049239027
0.11206839585349024
0.1245161883556719
0.11525600030930994
0.11680378254877251
0.1192810966289441
0.13620244865236855
0.1048222239287034
0.10000000000000003
0.096785571854924
0.09705716063717568
0.1059198958400349
0.13748825603247225
0.11107777493186066
0.13514203244185358
0.09090909090909091
0.13808620494238594
0.09195235203666877
0.1321328060995165
0.12485830315772367
0.1826567981262361
0.1295529812688175
0.11403616509968494
0.12167661479624259
0.10363693755739212
0.1379384055008224
0.10000000000000003
0.13793840550082243
0.10000000000000003
0.13576675824455592
0.09090909090909091
0.11312049137215871
0.11849757673176153
0.1281993584310018
0.21119119641597162
0.1011664354

0.11662028117577775
0.10619899638351604
0.10834695468811359
0.10834695468811359
0.09090909090909091
0.1111111111111111
0.09356669665111224
0.10000000000000003
0.15509289605942952
0.10954596006554464
0.10050340422364232
0.1176844394782706
0.12818636252726265
0.09090909090909091
0.08921496814384021
0.09356669665111224
0.10619899638351603
0.11662028117577775
0.11495098238979357
0.10223284485522251
0.10067731183412842
0.14483390034102633
0.11120157342934568
0.1467737846153181
0.10400082644628103
0.10321570294784585
0.09090909090909091
0.09356669665111224
0.10223284485522251
0.11495098238979357
0.10619899638351604
0.11662028117577775
0.10758727976676699
0.10000000000000003
0.10783859440931536
0.1423273333106151
0.136579476831415
0.15242812396283925
0.09421555904651323
0.10321570294784582
0.09421555904651323
0.1111111111111111
0.13002453585600907
0.12861493192677012
0.10156836910630455
0.14479406599726363
0.07929042984987043
0.11825502802959231
0.10505461079324721
0.10000000000000003
0.10505

0.08333333333333336
0.09090909090909091
0.09667102135571166
0.10536018122181957
0.09667102135571166
0.10536018122181957
0.11242632085981444
0.10261370400196075
0.08333333333333336
0.09090909090909091
0.09090909090909091
0.08333333333333336
0.09860769411773258
0.09860769411773258
0.08333333333333336
0.10000000000000003
0.09860769411773258
0.09860769411773258
0.08333333333333336
0.09090909090909091
0.09667102135571166
0.10536018122181957
0.09090909090909091
0.08333333333333336
0.08333333333333336
0.09090909090909091
0.10452579080253187
0.10452579080253187
0.09090909090909091
0.08333333333333336
0.10000000000000003
0.08333333333333336
0.08333333333333336
0.09090909090909091
0.09860769411773258
0.09860769411773258
0.08333333333333336
0.09090909090909091
0.10452579080253187
0.10452579080253187
0.09090909090909091
0.09090909090909091
0.09667102135571166
0.10536018122181957
0.08333333333333336
0.09090909090909091
0.09090909090909091
0.08333333333333336
0.08333333333333336
0.09090909090909091


0.10423304506144748
0.08333333333333336
0.09090909090909091
0.09090909090909091
0.08333333333333336
0.09090909090909091
0.08333333333333336
0.09090909090909091
0.08333333333333336
0.08333333333333336
0.09090909090909091
0.10423304506144748
0.10423304506144748
0.08333333333333336
0.09090909090909091
0.09090909090909091
0.08333333333333336
0.0982844243139433
0.0982844243139433
0.08333333333333336
0.09090909090909091
0.0982844243139433
0.0982844243139433
0.09090909090909091
0.08333333333333336
0.08333333333333336
0.09090909090909091
0.08333333333333336
0.09090909090909091
0.08333333333333336
0.09090909090909091
0.09090909090909091
0.08333333333333336
0.09090909090909091
0.08333333333333336
0.10423304506144748
0.10423304506144748
0.10423304506144748
0.10423304506144748
0.09090909090909091
0.08333333333333336
0.10423304506144748
0.10423304506144748
0.08333333333333336
0.09090909090909091
0.09090909090909091
0.08333333333333336
0.08333333333333336
0.09090909090909091
0.0982844243139433
0.098

Inversão da visualização:

In [54]:
def inver_show(g):
    cg = nx.complement(g)

    sn.reset_positions(cg)

    for n in g.nodes:
        g.nodes[n]['pos'] = cg.nodes[n]['pos']

    sn.show_graph(g)


inver_show(g)

Distribuição de *betweenness*:

In [55]:
betweenness = nx.betweenness_centrality(g)

df = pd.DataFrame({
    'betweenness': [betweenness[n] for n in g.nodes],
})

df.describe()

,betweenness
count,25.000000
mean,0.020870
std,0.002629
min,0.018395
25%,0.018395
50%,0.018395
75%,0.023551
max,0.023551


Simulação com visualização, quando apenas uma minoria dos nós busca reduzir a restrição de Burt, enquanto todos os outros buscam aumentar:

In [57]:
num_closers = int(FRAC_CLOSERS * NUM_NODES)

g = simulate(num_closers)

sn.reset_positions(g)

sn.show_graph(g)

1.1379560972955302e-07
2.9543148345188993e-07
0.1693222240537425
2.471487501614572e-07
2.3145790387699793e-07
3.606183531846118e-08
3.923913219080005e-08
1.1081730951824373e-07
8.914455926273147e-08
1.291417436480552e-07
2.093462360153116e-07
2.932234274056365e-07
1.5189608908625734e-07
0.16472400356724795
1.094810192288093e-06
1.144062340748611e-07
5.695220629724485e-08
5.08729440313614e-08
0.16351276841922036
5.528223705951721e-07
1.2680107040832998e-07
8.015165584351527e-08
0.16405882417246054
0.19117600953016226
1.4826561127060993e-07
1.9244471764259258e-07
0.1984152487736237
9.924926897503521e-08
4.6215459598573174e-08
1.4499633514040075e-07
8.729430011533933e-08
5.89256540340701e-08
5.168165377086715e-07
9.375355041097852e-08
6.279219395787738e-08
0.1731139573047665
1.218365252383505e-06
5.549442468208022e-08
1.506736981918682e-07
1.4698098722501024e-07
1.0244503084308724e-06
6.644672723837058e-08
2.2853399428276333e-08
5.46467495211153e-08
1.619039779693526e-07
5.983486688914213

8.331137845085598e-09
1.209708572927836e-08
2.2885799001868193e-08
0.1664315788797576
1.3811490962292106e-08
0.170098709098444
1.4591682328773194e-08
0.16362051155149462
1.1436875810277976e-08
0.16506565531845477
0.16442510662187443
9.88584850889523e-09
1.3771357324029785e-08
0.1686411129164103
1.2954938736288598e-08
0.16328255966370547
2.5474785396844037e-08
1.067540982610226e-08
1.240317189532112e-08
7.442950120988102e-09
1.2691675707979521e-08
1.890637907841101e-08
1.33432279109197e-08
8.691062097457326e-09
1.0389468419710204e-08
0.16808104884697236
8.46522256411315e-09
0.1636612016399467
1.2724230835834967e-08
1.5428867080415895e-08
9.719938013019913e-09
0.16656436766995303
1.2587852010149987e-08
1.4742992100468412e-08
2.4909183108465378e-08
1.1135036497117686e-08
1.522749396208027e-08
9.085055555821774e-09
1.3922099655021711e-08
1.4276870113144915e-08
1.4486962784819987e-08
4.7822765960965184e-08
1.2202765794434838e-08
9.253610714149305e-09
4.70629091684563e-08
1.1987140770670315e

1.035988754996876e-08
9.235327114905576e-09
7.719752369308632e-09
9.01693459336402e-09
0.16823745600191015
1.217638468234122e-08
1.1710449654540625e-08
6.8374820386221274e-09
0.16465261085686958
1.1445587687990244e-08
6.637155898005299e-09
0.1630247757695318
7.080569621871779e-09
1.1469678612941053e-08
7.925430376938962e-09
1.2212963188371672e-08
0.16798850399753631
0.15538541320322136
1.053075983858104e-08
1.0747284426839632e-08
9.568545571743436e-09
8.85235048302263e-09
7.084263043988914e-09
1.2667187784795559e-08
0.16596070050743264
0.1714613367085947
0.18191866049210678
7.248664764143046e-09
0.16286866414694728
0.15428044246898526
1.2488997544490976e-08
1.1741882748205678e-08
1.1012313987595395e-08
1.1564394098949726e-08
9.098591645947158e-09
0.15483995380916918
1.1549214754706705e-08
0.15376795420685208
9.111028780609664e-09
8.99657557255222e-09
7.708002110643167e-09
1.2576760962831836e-08
7.766541312115341e-09
9.968106651404325e-09
8.601073643471566e-09
1.1242438112605324e-08
1.2

0.16379558109028833
7.234965427847533e-09
0.17086946776652404
6.730491997256204e-09
6.9872569773050624e-09
1.0940304967948502e-08
0.1683023848552099
1.4591794773127511e-08
9.180939477623223e-09
8.891603113045099e-09
1.1630605662445924e-08
6.874830560024725e-09
8.917175602309524e-09
1.1487203187016576e-08
7.288254687334146e-09
9.634701546853934e-09
4.861687718888428e-09
1.8984036325454677e-08
1.2622720858562781e-08
6.7239455890395305e-09
7.35622123700091e-09
8.646204004369877e-09
8.626346685465537e-09
1.1649352924348456e-08
6.088991120812313e-09
1.1546777978525358e-08
7.336725713095566e-09
9.255312864929138e-09
1.0605422605841571e-08
0.15450927709102955
7.656450571696767e-09
1.1729846244032669e-08
1.3429199256741738e-08
0.15909606686411104
1.1979394820493072e-08
5.657445870824011e-09
1.0636214161643811e-08
6.797586205480589e-09
9.015297217087687e-09
0.168437442499253
0.1720090947981882
1.131918887435845e-08
1.1306559255345586e-08
1.1825371466467006e-08
0.17193171152702127
9.248318168248

6.401940359878134e-09
0.16795605917697207
9.263905835423094e-09
7.722966010337504e-09
7.531259841177974e-09
7.641389993522928e-09
7.706213500090381e-09
7.22452831738248e-09
7.353513669306003e-09
6.0670118727479965e-09
7.451401335444577e-09
7.77515571412796e-09
1.1624956355796453e-08
1.2474189010381226e-08
6.412135899289696e-09
0.1659466989723914
0.16546287894293515
0.15921921744547204
1.1901021304450288e-08
1.1453870768719482e-08
6.1892784123393934e-09
9.808443371766094e-09
6.403930865057422e-09
0.1733407969417309
0.16456080489717909
6.357999956092192e-09
6.490189170298178e-09
0.1674464279774736
7.666764508103757e-09
1.2243452789402441e-08
8.150693924086414e-09
0.16928958140365455
1.2317764738195493e-08
9.308812478960412e-09
1.2121325366373031e-08
9.251405538782123e-09
7.2331169981640844e-09
6.005040489310947e-09
9.249875697979655e-09
6.278088842351819e-09
1.937690692570706e-08
1.256893222283378e-08
5.2923056400488445e-09
0.16832194650820143
1.1588445643736777e-08
7.32910018617304e-09


1.1918627946166104e-08
5.126767068264121e-09
7.722130496345514e-09
7.542890036200929e-09
6.443838686981729e-09
0.162764942019902
1.894076945735294e-08
1.0002242526794369e-08
7.436940179839973e-09
5.040199849484187e-09
6.442637554139755e-09
0.16791227520654897
7.63614757754098e-09
1.1651825506585161e-08
7.585766698839129e-09
7.8232701118674e-09
9.440605045402888e-09
9.306276470892475e-09
7.750196652025819e-09
9.520238074420465e-09
4.483353778881989e-09
0.17212246738585987
8.045789008241462e-09
0.16526461083538388
0.1812146605866997
8.175042848075301e-09
5.8763130462565376e-09
4.95630042615514e-09
1.196992847517475e-08
7.791083794859716e-09
0.16899037245233267
0.17162327930960763
0.15810274007962527
6.553894291672736e-09
1.1663708584531995e-08
1.2035430016085471e-08
7.427299678215059e-09
6.037459030978942e-09
6.108517715085185e-09
7.719068333702013e-09
0.16407406708970876
5.433424103547433e-09
5.096771274334196e-09
9.629929074065876e-09
1.2051712066884375e-08
5.325095920556946e-09
1.2640

1.1713167575597015e-08
7.440955722404013e-09
7.755409656814203e-09
9.612840524355154e-09
1.153564246661425e-08
9.164525228166672e-09
1.1967947544351267e-08
7.66335136622004e-09
1.867189136155973e-08
6.462201105638927e-09
7.879515854281605e-09
0.16526461083538385
9.53402330602683e-09
6.172759217536355e-09
7.58239884707407e-09
9.470152941428273e-09
7.365403157702104e-09
9.211731535900916e-09
6.106913786685747e-09
7.680274635563204e-09
1.2091724872487448e-08
9.846793394780412e-09
9.300185826933706e-09
6.304949098399485e-09
0.15831844816933166
6.590962630299476e-09
4.483095529640169e-09
0.16630014424714698
7.653743847321858e-09
9.740823662233601e-09
7.437583714907683e-09
1.1699339788584236e-08
5.036919787508158e-09
7.2607769812303294e-09
0.1560873372093226
0.1648050112157745
7.460443953626445e-09
9.470152941428268e-09
8.06359964680962e-09
0.17882039936420885
7.928532256849524e-09
0.1708401414705961
5.376909419149486e-09
0.1593319619015431
1.175874179048276e-08
6.106645864512375e-09
6.17195

Distribuição de *betweenness*:

In [58]:
betweenness = nx.betweenness_centrality(g)

df = pd.DataFrame({
    'betweenness': [betweenness[n] for n in g.nodes],
})

df.describe()

,betweenness
count,25.000000
mean,0.007246
std,0.003091
min,0.002635
25%,0.004587
50%,0.006857
75%,0.009295
max,0.014326


Distribuição de *betweenness* dos *openers*:

In [59]:
betweenness = nx.betweenness_centrality(g)

df = pd.DataFrame({
    'betweenness': [betweenness[n] for n in g.nodes if g.nodes[n]['type'] == OPENER],
})

df.describe()

,betweenness
count,5.000000
mean,0.004550
std,0.001897
min,0.002635
25%,0.003450
50%,0.003952
75%,0.005208
max,0.007503


Distribuição de *betweenness* dos *closers*:

In [60]:
betweenness = nx.betweenness_centrality(g)

df = pd.DataFrame({
    'betweenness': [betweenness[n] for n in g.nodes if g.nodes[n]['type'] == CLOSER],
})

df.describe()

,betweenness
count,20.000000
mean,0.007921
std,0.002986
min,0.003091
25%,0.005585
50%,0.008293
75%,0.009880
max,0.014326
